In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV, LinearRegression, Lasso
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

import sys
sys.path.append('../src/')
from ffcDataCleaning.CodeBook import getCodeDescription

from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
gen_features = pd.read_csv('../output/imputing/.data/imputed.csv', index_col=0)
gen_features.head()

In [ ]:
specific_features = pd.read_csv('../output/imputing/.data/materialHardship_features.csv', index_col=0)
specific_features_total = pd.read_csv('../output/imputing/.data/materialHardshipTotal_features.csv', index_col=0)
specific_features = specific_features.join([specific_features_total])
specific_features.head()
print specific_features.shape
print specific_features.columns

In [ ]:
features = specific_features
#features = pd.concat([specific_features,gen_features], axis = 1)
print features.shape
features.head()

In [ ]:
data_dir = '../.data'
fp_train = '{}/train.csv'.format(data_dir)
df_test = pd.read_csv(fp_train,index_col=0)

In [ ]:
Y = df_test.materialHardship.dropna()
print Y.shape
idx = Y.index

S = StandardScaler()
idx_features = features.index
FEATURES_TRAIN = S.fit_transform(features)
X = FEATURES_TRAIN[idx-1,:]
print X.shape

In [ ]:
b = 500
reg = LassoCV(selection='random', cv=10, n_jobs=-1)
reg.fit(X[:b],Y[:b])

In [ ]:
from sklearn.feature_selection import SelectFromModel

ms = SelectFromModel(reg, threshold = 0.00001, prefit = True)
x_red = ms.transform(X)
impt_features = features.columns[ms.get_support()]

In [ ]:
print '\n'.join(['{:.<15s}{}'.format(x,getCodeDescription(x)) for x in impt_features])

In [ ]:
Y_pred = reg.predict(X[b:])
print mean_squared_error(Y[b:], Y_pred)
plt.hist(Y_pred, normed=True)
plt.hist(Y, normed=True)

In [ ]:
reg2 = LassoCV()
reg2.fit(x_red,Y)
Y_pred = reg2.predict(x_red[b:])
print mean_squared_error(Y[b:], Y_pred)
print reg2.alpha_

#### Best Score: 0.02415
#### Baseline Score: 0.02880

In [ ]:
scores = cross_val_score(LinearRegression(),x_red,Y,cv=10,scoring='neg_mean_squared_error',n_jobs=-1)
print abs(scores)
print scores.mean()

## Final Fit

1. Use all features -- reduced during selection.

In [ ]:
final_reg = LassoCV(selection='random', cv=10, n_jobs=-1)
final_reg.fit(X,Y)
Y_Final = final_reg.predict(FEATURES_TRAIN)

In [ ]:
Y_Final = pd.DataFrame(Y_Final, index = idx_features, columns=['materialHardship'])

In [ ]:
#sanity checks
print Y_Final.shape
print mean_squared_error(Y.values,Y_Final.loc[idx].values)
Y_Final.hist()
Y.hist()

In [ ]:
Y_Final.to_csv('../output/solutions/Solution1/materialHardship.csv')